#### Import Libraries

In [ ]:
# Import python packages
import streamlit as st
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import max,min,avg,call_function, month, dayofmonth, parse_json,split,substr,hour,replace,concat,col,lit,array_slice,array_agg,object_construct,to_time, date_add,to_date, current_date
from snowflake.snowpark.types import StringType,VariantType, DateType, IntegerType,DecimalType

import pandas as pd
import altair as alt



# Write directly to the app
st.title("Northern Trains Weather Data :train:")
st.write(
    """This app shows the weather that may effect Northern Trains).
    """
)

# Get the current credentials
session = get_active_session()

In [ ]:
st.markdown('#### A dataframe which shows all the train stations')
trains_latlon = session.table('NORTHERN_TRAINS_STATION_DATA.TESTING."StationLatLong"')
trains_latlon

#### Create a Postcode Sector Column

In [ ]:
trains_latlon = trains_latlon.with_column('POSTCODE_SPLIT',split(col('"Postcode"'),lit(' ')))
trains_latlon = trains_latlon.with_column('Postcode_outcode',col('POSTCODE_SPLIT')[0].astype(StringType()))
trains_latlon = trains_latlon.with_column('Postcode_inward_code',col('POSTCODE_SPLIT')[1].astype(StringType()))
trains_latlon = trains_latlon.with_column('Postcode_Sector',concat('POSTCODE_OUTCODE',lit('_'),substr(col('Postcode_inward_code'),1,1)))
trains_latlon = trains_latlon.drop('POSTCODE_SPLIT','POSTCODE_OUTCODE','POSTCODE_INWARD_CODE')
trains_latlon

In [ ]:
### load in hourly forecast
weather_hourly = session.table('POSTCODE_SECTOR_WEATHER_FORECASTS_PRIVATE_BETA.BD1_BETA."advanced_with_solar_hourly_view"')

### load in daily forecast

weather_daily = session.table('POSTCODE_SECTOR_WEATHER_FORECASTS_PRIVATE_BETA.BD1_BETA."advanced_with_solar_daily_view"')

weather_hourly_max = weather_hourly.agg(max('"Issued_at"').alias('MAX'))
weather_hourly = weather_hourly.join(weather_hourly_max,weather_hourly_max['MAX']==weather_hourly['"Issued_at"']).drop('MAX')

### find the latest weather forecast
weather_daily_max = weather_daily.agg(max('"Issued_at"').alias('MAX'))

### join the latest issue date to the dataset in order to filter it.
weather_daily = weather_daily.join(weather_daily_max,weather_daily_max['MAX']==weather_daily['"Issued_at"']).drop('MAX')


### repeat with the hourly dataset
weather_hourly = weather_hourly.join(trains_latlon,trains_latlon['POSTCODE_SECTOR']==weather_hourly['PC_SECT']).drop('PC_SECT')
weather_hourly = weather_hourly.with_column('VALIDITY_DATE',to_date('"Validity_date_and_time"'))

st.markdown('#### HOURLY DATA')
st.dataframe(weather_hourly.limit(10))

#### join with the the train station data by the PC_SECT field
weather_daily = weather_daily.join(trains_latlon,trains_latlon['POSTCODE_SECTOR']==weather_daily['PC_SECT']).drop('PC_SECT')

st.markdown('#### DAILY DATA')
st.dataframe(weather_daily.limit(10))

### Create a Human Readable Weather Forecast

#### STEP 1 - Create Data Filters

In [ ]:
station_filter = trains_latlon.select('"CrsCode"')
date_filter = weather_hourly.agg(max('VALIDITY_DATE').alias('MAX'),
                          min('VALIDITY_DATE').alias('MIN')).to_pandas()




selected_station = st.selectbox('Select Station:',station_filter)
selected_date = st.date_input('Select Date:',date_filter.MIN.iloc[0],date_filter.MIN.iloc[0],date_filter.MAX.iloc[0])

#### Create the Prompt

In [ ]:
todaypd = weather_daily.filter((col('"Validity_date"')==date_add(lit(selected_date),1))
                                     & (col('"CrsCode"')==selected_station)).to_pandas()

st.write(todaypd)
melt = pd.melt(todaypd)
melt['variable'] = melt['variable'].astype("string")
melt['value'] = melt['value'].astype("string")


##### create an object to feed into the LLM
object = session.create_dataframe(melt)
object = object.with_column('object',object_construct(col('"variable"'),col('"value"')))
    
object = object.select(array_agg('OBJECT').alias('OBJECT'))



prompt = object.select(concat(lit('Summarise the weather report in 300 words for today which includes relevant emojis to summarise the weather based on the following dataset'),
                    col('object').astype(StringType()),
                   lit('USE APPROPIATE MARKDOWN TO ENHANCE THE PRESENTATION. NO COMMENTS')).alias('PROMPT'))

st.code(prompt.to_pandas().PROMPT.iloc[0])



#### Run the LLM

In [ ]:
complete = prompt.select(call_function('snowflake.cortex.complete',lit('mixtral-8x7b'),col('PROMPT')))
st.write(complete.collect()[0][0])

In [ ]:

weather_filter_hour = weather_hourly.filter(col('"CrsCode"') == selected_station)
weather_filter_day = weather_daily.filter(col('"CrsCode"') == selected_station)

weather_filter_hour = weather_filter_hour.filter(col('VALIDITY_DATE')==selected_date)
#weather_filter_day = weather_filter_day.filter(col('"Validity_date"')==selected_date)
weather_filter_hour = weather_filter_hour.with_column('HOUR',hour('"Validity_date_and_time"'))


weather_filter_hour = weather_filter_hour.group_by('HOUR').agg(avg('"Feels_like_temperature"').alias('Feels Like'),
                                                              avg('"Screen_temperature"').alias('"Screen"'),
                                                               avg('"Screen_Dew_Point_Temperature"').alias('"Dew Point"'),
avg('"UV_index"').alias(' '),                                                        avg('"Relative_humidity"').alias('"Relative"'),                                                        avg('"Wind_speed"').alias('"Speed"'),
 avg('"Wind_gust"').alias('"Gust"'),).to_pandas()

#weather_filter_hour
st.markdown('#### 12 DAY DAILY FORECAST')

col1,col2 = st.columns(2)
with col1:
    st.markdown('##### TEMPERATURE')
#weather_filter_day

    st.line_chart(weather_filter_day.to_pandas(),x='Validity_date',y=['Max_temperature_day','Max_feels_like_temperature_day','Min_feels_like_temperature_night'])


with col2:
    st.markdown('##### HUMIDITY')
#weather_filter_day

    st.line_chart(weather_filter_day.to_pandas(),x='Validity_date',y=['Relative_humidity_Approx_Local_Midday','Relative_humidity_Approx_Local_Midnight'])

st.divider()

st.markdown('#### HOURLY VIEW')

col1,col2 = st.columns(2)

with col1:
    st.markdown('##### TEMPERATURE')
    st.line_chart(weather_filter_hour,x='HOUR',y=['Feels Like','Screen','Dew Point'])
with col2:
    st.markdown('##### UV INDEX')
    st.bar_chart(weather_filter_hour,x='HOUR',y=[' '])

with col1:
    st.markdown('##### WIND')
    st.line_chart(weather_filter_hour,x='HOUR',y=['Speed','Gust'])
with col2:
    st.markdown('##### HUMIDITY')
    st.bar_chart(weather_filter_hour,x='HOUR',y=['Relative'])

#### Historic Weather Patterns for the Chosen Key events

##### Select the event data generated earlier

In [ ]:
Events = session.table('BUILD_UK.DATA.EVENTS_IN_THE_NORTH').join_table_function('flatten',parse_json('EVENT_DATA')).select('VALUE')

Events = Events.select(col('VALUE')['NAME'].astype(StringType()).alias('NAME'),
              col('VALUE')['DESCRIPTION'].astype(StringType()).alias('DESCRIPTION')
              ,col('VALUE')['COLOR'].alias('COLOR')
              ,col('VALUE')['DATE'].astype(DateType()).alias('DATE'),col('VALUE')['CENTROID'].alias('CENTROID')).drop('VALUE')


####  Use Cortex to help return a postcode sector for the given co-ordinates

In [ ]:
Events = Events.with_column('SECTOR',call_function('SNOWFLAKE.CORTEX.COMPLETE',lit('mixtral-8x7b'), 
                                     concat(lit('Return the postcode sector for the following point:'),col('CENTROID').astype(StringType())
                                            ,lit('RETURN THER RESULT IN JSON FORMAT. DO NOT RETURN EXPLANATIONS.  only return JSON.'))))

Events = Events.with_column('SECTOR',parse_json('SECTOR')['postcode_sector'].astype(StringType()))
Events = Events.with_column('SECTOR',replace(col('SECTOR'),lit(' '),lit('_')))
Events

#### Load the daily weather forecast data

In [ ]:
historic_weather_forecasts = session.table('POSTCODE_SECTOR_WEATHER_FORECASTS_PRIVATE_BETA.BD1_BETA."advanced_with_solar_daily_view"')

#### Filter by Event

In [ ]:
selected_event = st.selectbox('Select Event:', Events.select('NAME'))

historic_weather_forecasts_events = historic_weather_forecasts.join(Events,(Events['SECTOR']==historic_weather_forecasts['PC_SECT']))

historic_weather_forecasts_events = historic_weather_forecasts_events.filter((col('NAME')==selected_event)& (to_date('"Issued_at"').between(date_add(current_date(),lit(-10)),current_date()))) 








source = historic_weather_forecasts_events.to_pandas()


def forecast_variations(measure):
    return alt.Chart(source).mark_line().encode(
    x='Validity_date:T',
    y=f'{measure}:Q',
    color=alt.Color('Issued_at:T',scale=alt.Scale(scheme='redyellowblue'))
)


col1,col2 = st.columns(2)

with col1:
    st.markdown('#### MAX TEMPERATURE FORECAST VARIATIONS')
    st.altair_chart(forecast_variations('Max_temperature_day'), use_container_width=True)
with col2:
    st.markdown('#### MIN TEMPERATURE FORECAST VARIATIONS')
    st.altair_chart(forecast_variations('Min_temperature_night'), use_container_width=True)

with col1:
    st.markdown('#### PROBABILITY OF RAIN DURING THE DAY')
    st.altair_chart(forecast_variations('Probability_of_Rain_day'), use_container_width=True)
with col2:
    st.markdown('#### PROBABILITY OF RAIN AT NIGHT')
    st.altair_chart(forecast_variations('Probability_of_Rain_night'), use_container_width=True)

with col1:
    st.markdown('#### PROBABILITY OF SUNSHINE DURING THE DAY')
    st.altair_chart(forecast_variations('Probability_of_Sunshine_day'), use_container_width=True)
with col2:
    st.markdown('#### PROBABILITY OF CLEAR SKIES AT NIGHT')
    st.altair_chart(forecast_variations('Probability_of_Clear_Skies_night'), use_container_width=True)